# Engery Charts API
*Insert Lambda Function Name*

In [ ]:
import json
import requests
import psycopg2
import boto3
import os
from datetime import datetime

In [ ]:
ENDPOINT = os.environ['ENDPOINT']
DB_NAME = os.environ['DB_NAME']
USERNAME = os.environ['USERNAME']
PASSWORD = os.environ['PASSWORD']
S3_BUCKET = os.environ['S3_BUCKET'] 
API_KEY = os.environ['API_KEY']

In [ ]:
def fetch_currency_data(source_currency="CHF", target_currencies="EUR"):
    """Fetch live exchange rates with customizable source and target currencies"""
    if not API_KEY:
        return {"error": "API key not found"}

    base_url = "https://api.apilayer.com/currency_data/"
    endpoint = "live"
    api_url = f"{base_url}{endpoint}?source={source_currency}&currencies={target_currencies}"
    headers = {"apikey": API_KEY}

    response = requests.get(api_url, headers=headers)
    return response.json() if response.status_code == 200 else {"error": response.text}

def store_currency_data_in_rds(currency_data):
    """Store live exchange rates in RDS"""
    try:
        print("Connecting to DB...")
        conn = psycopg2.connect(
            host=ENDPOINT,
            dbname=DB_NAME,
            user=USERNAME,
            password=PASSWORD
        )
        cur = conn.cursor()
        conn.set_session(autocommit=True)
        print("DB connection successful.")

        # Define the table schema and columns
        table_name = "currency_data"
        columns = {
            "timestamp": "TIMESTAMP NOT NULL",
            "source_currency": "VARCHAR(10)",
            "target_currency": "VARCHAR(10)",
            "exchange_rate": "FLOAT"
        }

        # Create table dynamically if it doesn't exist
        create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} ("
        create_table_query += ", ".join([f"{col} {datatype}" for col, datatype in columns.items()])
        create_table_query += ", PRIMARY KEY (timestamp, source_currency, target_currency));"

        cur.execute(create_table_query)

        insert_query = f"""
        INSERT INTO {table_name} ({', '.join(columns.keys())})
        VALUES (%s, %s, %s, %s)
        ON CONFLICT (timestamp, source_currency, target_currency) DO NOTHING;
        """

        now = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]  # Format to 'YYYY-MM-DD HH:MM:SS.mmm'

        for pair, rate in currency_data.get("quotes", {}).items():
            source_currency, target_currency = pair[:3], pair[3:]
            cur.execute(
                insert_query,
                (now, source_currency, target_currency, rate)
            )

        cur.close()
        conn.close()

    except psycopg2.Error as e:
        print("Database error:", e)

def convert_unix_to_iso(timestamp):
    """Convert Unix timestamp to ISO 8601 format"""
    return datetime.utcfromtimestamp(timestamp).isoformat()

def preprocess_currency_data(data):
    """Fix timestamps in the currency data JSON"""
    if "timestamp" in data:
        data["timestamp"] = convert_unix_to_iso(data["timestamp"])
    return data

def store_json_to_s3(currency_data):
    """Store the JSON data to S3 bucket in a structured way"""
    s3 = boto3.client('s3')
    today = str(datetime.today().date())
    foldername = "currency_data"
    key = f"{foldername}/{foldername}_{today}.json"

    processed_data = preprocess_currency_data(currency_data)

    s3.put_object(
        Bucket=S3_BUCKET,
        Key=key,
        Body=json.dumps(processed_data, indent=2),
        ContentType='application/json'
    )

def lambda_handler(event, context):
    """AWS Lambda Entry Point"""
    
    # Fetch live data
    currency_data = fetch_currency_data()
    
    if "error" in currency_data:
        return {
            "statusCode": 400,
            "body": json.dumps({"error": currency_data["error"]}, indent=2)
        }
    
    store_currency_data_in_rds(currency_data)
    store_json_to_s3(currency_data)

    return {
        "statusCode": 200,
        "body": json.dumps({
            "message": "Live exchange rates stored successfully in RDS and S3.",
            "s3_path": f"s3://{S3_BUCKET}/currency_data/currency_rates_{datetime.today().strftime('%Y-%m-%d')}.json",
            "currency_rates": currency_data
        }, indent=2)
    }